# Buzzer Finder Obos

## Pemprosesan Data

In [1]:
import pandas as pd
import re
from operator import itemgetter
import ast
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network
import networkx.algorithms.community as nx_com

In [3]:
#Baca file csv
df = pd.read_csv('not_preposessing.csv', index_col = [0])

df.head()

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1785006273387438334,Mon Apr 29 18:00:34 +0000 2024,0,@vidio kenapa match lavani vs PBS ga bisa dipu...,1785006273387438334,NaN,vidio,in,"Di TanahAirku, Indonesia",0,1,0,https://twitter.com/Widya_skb/status/178500627...,372755145,Widya_skb
1,1784987684961968170,Mon Apr 29 16:52:54 +0000 2024,0,@vidio ga mau ga ngasih solusi jg.. solusinya ...,1784989247428288792,NaN,vidio,in,JC (1) 😇,0,0,0,https://twitter.com/DefVl06/status/17849892474...,2449483950,DefVl06
2,1784942779476623424,Mon Apr 29 13:48:15 +0000 2024,0,@vidio nyiarin indo vs usbek gk??,1784942779476623424,NaN,vidio,in,"Denpasar, Jakarta",0,2,0,https://twitter.com/radityaputra_/status/17849...,86374232,radityaputra_
3,1784719150105219270,Mon Apr 29 10:51:29 +0000 2024,0,@vidio Kecewa banget udah berbayar tapi gak bi...,1784898292716810490,NaN,vidio,in,"Metro Barat, Indonesia",0,0,0,https://twitter.com/nungkiki/status/1784898292...,183950036,nungkiki
4,1784870266088063164,Mon Apr 29 09:59:34 +0000 2024,0,@vidio Ohhhh di court 4 ternyata ga ada kamera...,1784885227808006416,NaN,vidio,in,NaN,0,0,0,https://twitter.com/PMskk15/status/17848852278...,1304725798050381824,PMskk15


## Pembuatan Struktur Data Baru

Langkah ini dilakukan untuk membuat struktur data baru menjadi source dan target. Source data didapatkan dari username yang membuat tweet, dan target didapatkan dari tag/mention @ yang ada pada suatu tweet.

In [11]:
import pandas as pd
import re
import os

# Baca data dari file CSV awal
try:
    df = pd.read_csv('not_preposessing.csv')
except FileNotFoundError:
    print("File 'ekonomi.csv' tidak ditemukan. Pastikan file tersebut ada di direktori yang sama dengan skrip ini.")
    exit()

# List untuk menyimpan tweet yang mengandung tag '@' dan usernamenya
tagged_tweets = []

# Ekstraksi tweet yang mengandung tag '@' dan usernamenya
for index, row in df.iterrows():
    full_text = row['full_text']
    username = row['username']

    # Gunakan ekspresi reguler untuk mencari tag '@' dan usernamenya
    mentions = re.findall(r'@(\w+)', full_text)

    # Jika ada tag '@', tambahkan ke list tagged_tweets
    if mentions:
        for mention in mentions:
            tagged_tweets.append({'source': username, 'target': mention})

# Buat DataFrame dari list tagged_tweets
result_df = pd.DataFrame(tagged_tweets)

# Print jumlah data
print(f"Jumlah data yang terbentuk: {len(result_df)}")

# Menghitung jumlah node unik
unique_nodes = set(result_df['source']).union(set(result_df['target']))
print(f"Jumlah node yang terbentuk: {len(unique_nodes)}")

# Tentukan nama file output
output_file = 'ekonomi_selected1.csv'

# Export hasil akhir ke dalam file CSV dengan delimiter ;
result_df.to_csv(output_file, index=False, sep=';')

# Cek apakah file berhasil disimpan
if os.path.exists(output_file):
    print(f"File '{output_file}' berhasil disimpan.")
else:
    print(f"File '{output_file}' tidak berhasil disimpan.")

# Baca file CSV hasil ekstraksi
try:
    df_selected = pd.read_csv(output_file, delimiter=';')
    print(df_selected.head())
    
    # Simpan dengan nama ekonomi_sctg
    final_output_file = 'not_prepos_sctg.csv'
    df_selected.to_csv(final_output_file, index=False)

    # Cek apakah file akhir berhasil disimpan
    if os.path.exists(final_output_file):
        print(f"File '{final_output_file}' berhasil disimpan.")
    else:
        print(f"File '{final_output_file}' tidak berhasil disimpan.")
except FileNotFoundError:
    print(f"File '{output_file}' tidak ditemukan untuk dibaca.")

Jumlah data yang terbentuk: 1324
Jumlah node yang terbentuk: 556
File 'ekonomi_selected1.csv' berhasil disimpan.
          source         target
0      Widya_skb          vidio
1        DefVl06          vidio
2        DefVl06  LavaniForever
3  radityaputra_          vidio
4       nungkiki          vidio
File 'not_prepos_sctg.csv' berhasil disimpan.


In [21]:
import pandas as pd
import re
import os

# Baca data dari file CSV awal
try:
    df = pd.read_csv('ekonomi.csv')
except FileNotFoundError:
    print("File 'ekonomi.csv' tidak ditemukan. Pastikan file tersebut ada di direktori yang sama dengan skrip ini.")
    exit()

# List untuk menyimpan tweet yang mengandung tag '@' dan usernamenya
tagged_tweets = []

# Ekstraksi tweet yang mengandung tag '@' dan usernamenya
for index, row in df.iterrows():
    full_text = row['full_text']
    username = row['username']
    in_reply_to = row['in_reply_to_screen_name']

    # Gunakan ekspresi reguler untuk mencari tag '@' dan usernamenya
    mentions = re.findall(r'@(\w+)', full_text)

    # Jika ada tag '@', tambahkan ke list tagged_tweets
    if mentions:
        for mention in mentions:
            tagged_tweets.append({'source': username, 'target': mention})

    # Tambahkan juga baris untuk in_reply_to_screen_name jika ada
    if pd.notna(in_reply_to):
        tagged_tweets.append({'source': username, 'target': in_reply_to})

# Buat DataFrame dari list tagged_tweets
result_df = pd.DataFrame(tagged_tweets)

# Print jumlah data
print(f"Jumlah data yang terbentuk: {len(result_df)}")

# Menghitung jumlah node unik
unique_nodes = set(result_df['source']).union(set(result_df['target']))
print(f"Jumlah node yang terbentuk: {len(unique_nodes)}")

# Tentukan nama file output
output_file = 'ekonomi_selected1.csv'

# Export hasil akhir ke dalam file CSV dengan delimiter ;
result_df.to_csv(output_file, index=False)

# Cek apakah file berhasil disimpan
if os.path.exists(output_file):
    print(f"File '{output_file}' berhasil disimpan.")
else:
    print(f"File '{output_file}' tidak berhasil disimpan.")

# Baca file CSV hasil ekstraksi
try:
    df_selected = pd.read_csv(output_file)
    print(df_selected.head())
    
    # Simpan dengan nama ekonomi_sctg
    final_output_file = 'ekonomi_sctg_rpl.csv'
    df_selected.to_csv(final_output_file, index=False)

    # Cek apakah file akhir berhasil disimpan
    if os.path.exists(final_output_file):
        print(f"File '{final_output_file}' berhasil disimpan.")
    else:
        print(f"File '{final_output_file}' tidak berhasil disimpan.")
except FileNotFoundError:
    print(f"File '{output_file}' tidak ditemukan untuk dibaca.")

Jumlah data yang terbentuk: 2041
Jumlah node yang terbentuk: 1370
File 'ekonomi_selected1.csv' berhasil disimpan.
          source         target
0       HIMNAFOR  wagyuwxdharto
1       HIMNAFOR  wagyuwxdharto
2       HIMNAFOR  sensofelation
3       HIMNAFOR  sensofelation
4  H4T14K4LN4L42   gibran_tweet
File 'ekonomi_sctg_rpl.csv' berhasil disimpan.


## Penghapusan pada data yang duplikat

In [12]:
import pandas as pd
import re
import os

# Baca data dari file CSV awal
try:
    df = pd.read_csv('not_preposessing.csv')
except FileNotFoundError:
    print("File 'ekonomi.csv' tidak ditemukan. Pastikan file tersebut ada di direktori yang sama dengan skrip ini.")
    exit()

# List untuk menyimpan tweet yang mengandung tag '@' dan usernamenya
tagged_tweets = []

# Ekstraksi tweet yang mengandung tag '@' dan usernamenya
for index, row in df.iterrows():
    full_text = row['full_text']
    username = row['username']

    # Gunakan ekspresi reguler untuk mencari tag '@' dan usernamenya
    mentions = re.findall(r'@(\w+)', full_text)

    # Jika ada tag '@', tambahkan ke list tagged_tweets
    if mentions:
        for mention in mentions:
            tagged_tweets.append({'source': username, 'target': mention})

# Buat DataFrame dari list tagged_tweets
result_df = pd.DataFrame(tagged_tweets)

# Tambahkan kolom 'weight' yang menghitung frekuensi kemunculan setiap pasangan (source, target)
result_df['weight'] = result_df.groupby(['source', 'target'])['source'].transform('count')

# Menghapus duplikat baris karena kita hanya membutuhkan satu baris per pasangan (source, target)
result_df = result_df.drop_duplicates()

# Urutkan DataFrame berdasarkan kolom 'weight' secara menurun (descending)
result_df = result_df.sort_values(by='weight', ascending=False)

# Print jumlah data
print(f"Jumlah data yang terbentuk: {len(result_df)}")

# Menghitung jumlah node unik
unique_nodes = set(result_df['source']).union(set(result_df['target']))
print(f"Jumlah node yang terbentuk: {len(unique_nodes)}")

# Tentukan nama file output
output_file = 'ekonomi_selected1.csv'

# Export hasil akhir ke dalam file CSV dengan delimiter ;
result_df.to_csv(output_file, index=False)

# Cek apakah file berhasil disimpan
if os.path.exists(output_file):
    print(f"File '{output_file}' berhasil disimpan.")
else:
    print(f"File '{output_file}' tidak berhasil disimpan.")

# Baca file CSV hasil ekstraksi
try:
    df_selected = pd.read_csv(output_file)
    print(df_selected.head())
    
    # Simpan dengan nama ekonomi_sctg
    final_output_file = 'not_prepos_sctg.csv'
    df_selected.to_csv(final_output_file, index=False)

    # Cek apakah file akhir berhasil disimpan
    if os.path.exists(final_output_file):
        print(f"File '{final_output_file}' berhasil disimpan.")
    else:
        print(f"File '{final_output_file}' tidak berhasil disimpan.")
except FileNotFoundError:
    print(f"File '{output_file}' tidak ditemukan untuk dibaca.")

Jumlah data yang terbentuk: 949
Jumlah node yang terbentuk: 556
File 'ekonomi_selected1.csv' berhasil disimpan.
        source target  weight
0     Indosiar  vidio      16
1     arifsidy  vidio      11
2  herisantiko  vidio       9
3    xzibit017  vidio       8
4  anggpermadi  vidio       6
File 'not_prepos_sctg.csv' berhasil disimpan.


## Perhitungan Betweenness Centrality, Eigenvector Centrality serta normalisasi

In [5]:
import pandas as pd
import re
import networkx as nx
from sklearn.preprocessing import MinMaxScaler

# Baca data dari file CSV
df = pd.read_csv('israel-iran_selected1.csv', delimiter=';')

# List untuk menyimpan data dengan atribut weight
weighted_data = []

# Atribut weight dasar
base_weight = 1

# Ekstraksi data dan hitung weight
for index, row in df.iterrows():
    source = row['source']
    target = row['target']

    # Cari apakah data dengan source dan target yang sama sudah ada di weighted_data
    existing_data_index = None
    for i, data in enumerate(weighted_data):
        if data['source'] == source and data['target'] == target:
            existing_data_index = i
            break

    if existing_data_index is not None:
        # Jika data sudah ada, tambahkan 1 pada atribut weight
        weighted_data[existing_data_index]['weight'] += 1
    else:
        # Jika data belum ada, tambahkan data baru dengan weight dasar
        weighted_data.append({'source': source, 'target': target, 'weight': base_weight})

# Buat DataFrame dari list weighted_data
weighted_df = pd.DataFrame(weighted_data)

# Buat graf dari DataFrame
G = nx.from_pandas_edgelist(weighted_df, 'source', 'target', ['weight'])

# Hitung betweenness centrality
bc_values = nx.betweenness_centrality(G)

# Hitung eigenvector centrality
ev_values = nx.eigenvector_centrality(G)

# Buat DataFrame dari hasil betweenness centrality
bc_df = pd.DataFrame(list(bc_values.items()), columns=['account_name', 'BC'])

# Urutkan DataFrame berdasarkan kolom 'BC' secara descending
bc_df = bc_df.sort_values(by='BC', ascending=False)

# Buat DataFrame dari hasil eigenvector centrality
ev_df = pd.DataFrame(list(ev_values.items()), columns=['account_name', 'EV'])

# Urutkan DataFrame berdasarkan kolom 'EV' secara descending
ev_df = ev_df.sort_values(by='EV', ascending=False)

# Normalisasi menggunakan Min-Max Scaling untuk kolom 'EV'
scaler_ev = MinMaxScaler(feature_range=(0, 1))
ev_df['norm_EV'] = ev_df['EV'].rank() / len(ev_df)

# Normalisasi menggunakan Min-Max Scaling untuk kolom 'BC'
scaler_bc = MinMaxScaler()
bc_df['norm_BC'] = bc_df['BC'].rank() / len(bc_df)

# Gabungkan DataFrame bc_df dan ev_df berdasarkan kolom 'account_name'
result_df = pd.merge(bc_df, ev_df, on='account_name', how='outer')

# Susun kolom sesuai dengan urutan yang diinginkan
result_df = result_df[['account_name', 'BC', 'EV', 'norm_BC', 'norm_EV']]

# Cetak hasil
print(result_df)

# Export hasil akhir ke dalam file CSV dengan delimiter ';'
result_df.to_csv('hasilisrael-iran_combined_normalized.csv', index=False, sep=';')

         account_name   BC            EV   norm_BC   norm_EV
0        007AliSohrab  0.0  1.096318e-32  0.359957  0.459105
1            0xDamien  0.0  1.316192e-44  0.359957  0.093504
2          103sTaR103  0.0  3.084112e-08  0.359957  0.593184
3              108SAR  0.0  1.657983e-40  0.359957  0.222684
4         121Clueless  0.0  7.417330e-06  0.359957  0.737913
...               ...  ...           ...       ...       ...
4690        zerosum24  0.0  1.316192e-44  0.359957  0.093504
4691        zewaddler  0.0  9.441364e-36  0.359957  0.393291
4692   ziggynutsmcgee  0.0  8.531090e-35  0.359957  0.429180
4693  zogbystrategies  0.0  5.730290e-05  0.359957  0.822684
4694  zuccheroet26185  0.0  1.316192e-44  0.359957  0.093504

[4695 rows x 5 columns]


## Perhitungan Final Measure 

Perhitungan final measure dilakukan untuk mencari rangking dari 10 buzzer teratas. Final measure didapatkan berdasarkan penjumlahan nilai betweenness centrality dan juga eigenvector centrality

In [14]:
import pandas as pd
import re
import networkx as nx
from sklearn.preprocessing import MinMaxScaler

# Baca data dari file CSV
df = pd.read_csv('not_prepos_sctg.csv')

# List untuk menyimpan data dengan atribut weight
weighted_data = []

# Atribut weight dasar
base_weight = 1

# Ekstraksi data dan hitung weight
for index, row in df.iterrows():
    source = row['source']
    target = row['target']

    # Cari apakah data dengan source dan target yang sama sudah ada di weighted_data
    existing_data_index = None
    for i, data in enumerate(weighted_data):
        if data['source'] == source and data['target'] == target:
            existing_data_index = i
            break

    if existing_data_index is not None:
        # Jika data sudah ada, tambahkan 1 pada atribut weight
        weighted_data[existing_data_index]['weight'] += 1
    else:
        # Jika data belum ada, tambahkan data baru dengan weight dasar
        weighted_data.append({'source': source, 'target': target, 'weight': base_weight})

# Buat DataFrame dari list weighted_data
weighted_df = pd.DataFrame(weighted_data)

# Buat graf dari DataFrame
G = nx.from_pandas_edgelist(weighted_df, 'source', 'target', ['weight'])

# Hitung betweenness centrality
bc_values = nx.betweenness_centrality(G)

# Hitung eigenvector centrality
ev_values = nx.eigenvector_centrality(G)

# Buat DataFrame dari hasil betweenness centrality
bc_df = pd.DataFrame(list(bc_values.items()), columns=['account_name', 'BC'])

# Urutkan DataFrame berdasarkan kolom 'BC' secara descending
bc_df = bc_df.sort_values(by='BC', ascending=False)

# Buat DataFrame dari hasil eigenvector centrality
ev_df = pd.DataFrame(list(ev_values.items()), columns=['account_name', 'EV'])

# Urutkan DataFrame berdasarkan kolom 'EV' secara descending
ev_df = ev_df.sort_values(by='EV', ascending=False)

# Normalisasi menggunakan Min-Max Scaling untuk kolom 'EV'
scaler_ev = MinMaxScaler(feature_range=(0, 1))
ev_df['norm_EV'] = ev_df['EV'].rank() / len(ev_df)

# Normalisasi menggunakan Min-Max Scaling untuk kolom 'BC'
scaler_bc = MinMaxScaler()
bc_df['norm_BC'] = bc_df['BC'].rank() / len(bc_df)

# Gabungkan DataFrame bc_df dan ev_df berdasarkan kolom 'account_name'
result_df = pd.merge(bc_df, ev_df, on='account_name', how='outer')

# Susun kolom sesuai dengan urutan yang diinginkan
result_df = result_df[['account_name', 'BC', 'EV', 'norm_BC', 'norm_EV']]

# Buat atribut baru 'final_measure' yang berisikan rata-rata dari 'norm_BC' dan 'norm_EV'
result_df['final_measure'] = (result_df['norm_BC'] + result_df['norm_EV']) / 2

# Urutkan DataFrame berdasarkan kolom 'final_measure' secara descending
result_df = result_df.sort_values(by='final_measure', ascending=False)

# Cetak hasil
print(result_df)

# Export hasil akhir ke dalam file CSV dengan delimiter ';'
result_df.to_csv('not_preposessing_buzzer.csv', index=False)

       account_name        BC        EV   norm_BC   norm_EV  final_measure
526           vidio  0.989541  0.693959  1.000000  1.000000       1.000000
231   bhulukhuduktv  0.008194  0.068582  0.982014  0.996403       0.989209
145            SCTV  0.004504  0.054229  0.960432  0.994604       0.977518
20         AulyaUma  0.004176  0.039683  0.951439  0.971223       0.961331
468        rizzk_aa  0.004176  0.039683  0.951439  0.971223       0.961331
..              ...       ...       ...       ...       ...            ...
271      dondipta93  0.000000  0.001500  0.353417  0.017986       0.185701
295  gopayindonesia  0.000000  0.001500  0.353417  0.017986       0.185701
245     bukanhoaxxx  0.000000  0.001500  0.353417  0.017986       0.185701
455      realmadrid  0.000000  0.001500  0.353417  0.017986       0.185701
9     AdjieSanPutro  0.000000  0.001500  0.353417  0.017986       0.185701

[556 rows x 6 columns]


## Daftar Top 10 Buzzer

In [17]:
import pandas as pd
import re
import networkx as nx
from sklearn.preprocessing import MinMaxScaler

# Baca data dari file CSV
df = pd.read_csv('not_prepos_sctg.csv')

# List untuk menyimpan data dengan atribut weight
weighted_data = []

# Atribut weight dasar
base_weight = 1

# Ekstraksi data dan hitung weight
for index, row in df.iterrows():
    source = row['source']
    target = row['target']

    # Cari apakah data dengan source dan target yang sama sudah ada di weighted_data
    existing_data_index = None
    for i, data in enumerate(weighted_data):
        if data['source'] == source and data['target'] == target:
            existing_data_index = i
            break

    if existing_data_index is not None:
        # Jika data sudah ada, tambahkan 1 pada atribut weight
        weighted_data[existing_data_index]['weight'] += 1
    else:
        # Jika data belum ada, tambahkan data baru dengan weight dasar
        weighted_data.append({'source': source, 'target': target, 'weight': base_weight})

# Buat DataFrame dari list weighted_data
weighted_df = pd.DataFrame(weighted_data)

# Buat graf dari DataFrame
G = nx.from_pandas_edgelist(weighted_df, 'source', 'target', ['weight'])

# Hitung betweenness centrality
bc_values = nx.betweenness_centrality(G)

# Hitung eigenvector centrality
ev_values = nx.eigenvector_centrality(G)

# Buat DataFrame dari hasil betweenness centrality
bc_df = pd.DataFrame(list(bc_values.items()), columns=['account_name', 'BC'])

# Urutkan DataFrame berdasarkan kolom 'BC' secara descending
bc_df = bc_df.sort_values(by='BC', ascending=False)

# Buat DataFrame dari hasil eigenvector centrality
ev_df = pd.DataFrame(list(ev_values.items()), columns=['account_name', 'EV'])

# Urutkan DataFrame berdasarkan kolom 'EV' secara descending
ev_df = ev_df.sort_values(by='EV', ascending=False)

# Normalisasi menggunakan Min-Max Scaling untuk kolom 'EV'
scaler_ev = MinMaxScaler(feature_range=(0, 1))
ev_df['norm_EV'] = ev_df['EV'].rank() / len(ev_df)

# Normalisasi menggunakan Min-Max Scaling untuk kolom 'BC'
scaler_bc = MinMaxScaler()
bc_df['norm_BC'] = bc_df['BC'].rank() / len(bc_df)

# Gabungkan DataFrame bc_df dan ev_df berdasarkan kolom 'account_name'
result_df = pd.merge(bc_df, ev_df, on='account_name', how='outer')

# Susun kolom sesuai dengan urutan yang diinginkan
result_df = result_df[['account_name', 'BC', 'EV', 'norm_BC', 'norm_EV']]

# Buat atribut baru 'final_measure' yang berisikan rata-rata dari 'norm_BC' dan 'norm_EV'
result_df['final_measure'] = (result_df['norm_BC'] + result_df['norm_EV']) / 2

# Urutkan DataFrame berdasarkan kolom 'final_measure' secara descending
result_df = result_df.sort_values(by='final_measure', ascending=False)

# Ambil top 10 nilai 'final_measure'
top_10_final_measure = result_df.head(10)

# Tampilkan tabel dengan urutan nomor dan account_name
top_10_final_measure_table = top_10_final_measure[['account_name']].reset_index(drop=True)
top_10_final_measure_table.index += 1

# Cetak tabel
print(top_10_final_measure_table)

     account_name
1           vidio
2   bhulukhuduktv
3            SCTV
4        AulyaUma
5        rizzk_aa
6    itsmeninaaja
7     JCIndonesia
8   KABAR_CHELSEA
9         albenft
10       Indosiar


## Graf Social Network Analysis

In [4]:
pip install pyvis

^C
Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---

Graf dibuat berdasarkan data source dan target yang sudah dibuat sebelumnya. Selanjutnya pada graf juga ditampilkan 10 Top Buzzer dan juga 10 Buzzer yang termasuk kedalam list akan divisualisasikan dengan node yang lebih besar, sehingga mudah di identifikasi.

In [3]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
from sklearn.preprocessing import MinMaxScaler

# Read data from the CSV file
df = pd.read_csv('not_prepos_sctg.csv')

# List untuk menyimpan data dengan atribut weight
weighted_data = []

# Atribut weight dasar
base_weight = 1

# Ekstraksi data dan hitung weight
for index, row in df.iterrows():
    source = row['source']
    target = row['target']

    # Cari apakah data dengan source dan target yang sama sudah ada di weighted_data
    existing_data_index = None
    for i, data in enumerate(weighted_data):
        if data['source'] == source and data['target'] == target:
            existing_data_index = i
            break

    if existing_data_index is not None:
        # Jika data sudah ada, tambahkan 1 pada atribut weight
        weighted_data[existing_data_index]['weight'] += 1
    else:
        # Jika data belum ada, tambahkan data baru dengan weight dasar
        weighted_data.append({'source': source, 'target': target, 'weight': base_weight})

# Buat DataFrame dari list weighted_data
weighted_df = pd.DataFrame(weighted_data)

# Buat graf dari DataFrame
G = nx.from_pandas_edgelist(weighted_df, 'source', 'target', ['weight'])

# Hitung betweenness centrality
bc_values = nx.betweenness_centrality(G)

# Hitung eigenvector centrality
ev_values = nx.eigenvector_centrality(G)

# Buat DataFrame dari hasil betweenness centrality
bc_df = pd.DataFrame(list(bc_values.items()), columns=['account_name', 'BC'])

# Urutkan DataFrame berdasarkan kolom 'BC' secara descending
bc_df = bc_df.sort_values(by='BC', ascending=False)

# Buat DataFrame dari hasil eigenvector centrality
ev_df = pd.DataFrame(list(ev_values.items()), columns=['account_name', 'EV'])

# Urutkan DataFrame berdasarkan kolom 'EV' secara descending
ev_df = ev_df.sort_values(by='EV', ascending=False)

# Normalisasi menggunakan Min-Max Scaling untuk kolom 'EV'
scaler_ev = MinMaxScaler(feature_range=(0, 1))
ev_df['norm_EV'] = ev_df['EV'].rank() / len(ev_df)

# Normalisasi menggunakan Min-Max Scaling untuk kolom 'BC'
scaler_bc = MinMaxScaler()
bc_df['norm_BC'] = bc_df['BC'].rank() / len(bc_df)

# Gabungkan DataFrame bc_df dan ev_df berdasarkan kolom 'account_name'
result_df = pd.merge(bc_df, ev_df, on='account_name', how='outer')

# Susun kolom sesuai dengan urutan yang diinginkan
result_df = result_df[['account_name', 'BC', 'EV', 'norm_BC', 'norm_EV']]

# Buat atribut baru 'final_measure' yang berisikan rata-rata dari 'norm_BC' dan 'norm_EV'
result_df['final_measure'] = (result_df['norm_BC'] + result_df['norm_EV']) / 2

# Urutkan DataFrame berdasarkan kolom 'final_measure' secara descending
result_df = result_df.sort_values(by='final_measure', ascending=False)

# Ambil top 10 nilai 'final_measure'
top_10_final_measure = result_df.head(10)

# Tampilkan tabel dengan urutan nomor dan account_name
top_10_final_measure_table = top_10_final_measure[['account_name']].reset_index(drop=True)
top_10_final_measure_table.index += 1

# Create a networkx graph
G = nx.from_pandas_edgelist(df, 'source', 'target')

# Ekstrak top 10 account names dari DataFrame
top_account_names = top_10_final_measure['account_name'].tolist()

# Define the scale factor for node sizes
scale_factor_top = 30  # Adjust this value as needed
scale_factor_default = 15  # Adjust this value as needed for default nodes

# Create a pyvis network object
graph = Network(notebook=True)

# Add nodes and edges to the pyvis graph with specific size
for node in G.nodes:
    size = scale_factor_top if node in top_account_names else scale_factor_default
    graph.add_node(node, size=size)

for edge in G.edges:
    graph.add_edge(edge[0], edge[1])

# Set default options
options = {
    "nodes": {
        "shape": 'dot',
        "font": {
            "size": 12
        }
    }
}

# Convert options to a JSON-formatted string
options_str = str(options).replace("'", '"')

# Cetak tabel top 10 account names
print(top_10_final_measure_table)

# Show the graph with options
graph.set_options(options_str)
graph.show('graph.html') 

ModuleNotFoundError: No module named 'pyvis'

In [ ]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
from sklearn.preprocessing import MinMaxScaler

# Baca data dari file CSV
df = pd.read_csv('pemilu500_weighted.csv', delimiter=';')

# Membuat list untuk menyimpan data dengan atribut weight
weighted_data = []

# Atribut weight dasar
base_weight = 1

# Ekstraksi data dan hitung weight
for index, row in df.iterrows():
    source = row['source']
    target = row['target']

    # Cek apakah sudah ada data dengan source dan target yang sama di weighted_data
    existing_data_index = None
    for i, data in enumerate(weighted_data):
        if data['source'] == source and data['target'] == target:
            existing_data_index = i
            break

    if existing_data_index is not None:
        # Jika sudah ada, tambahkan weight
        weighted_data[existing_data_index]['weight'] += 1
    else:
        # Jika belum ada, tambahkan data baru dengan weight dasar
        weighted_data.append({'source': source, 'target': target, 'weight': base_weight})

# Buat DataFrame dari weighted_data
weighted_df = pd.DataFrame(weighted_data)

# Buat graf dari DataFrame
G = nx.from_pandas_edgelist(weighted_df, 'source', 'target', ['weight'])

# Hitung betweenness centrality dan eigenvector centrality
bc_values = nx.betweenness_centrality(G)
ev_values = nx.eigenvector_centrality(G)

# Gabungkan hasil dan normalisasi
centrality_df = pd.DataFrame({
    'account_name': list(bc_values.keys()),
    'BC': list(bc_values.values()),
    'EV': list(ev_values.values())
})

centrality_df['norm_BC'] = MinMaxScaler().fit_transform(centrality_df[['BC']])
centrality_df['norm_EV'] = MinMaxScaler().fit_transform(centrality_df[['EV']])
centrality_df['final_measure'] = (centrality_df['norm_BC'] + centrality_df['norm_EV']) / 2

# Ambil 10 nilai tertinggi dari 'final_measure'
top_10_final_measure = centrality_df.sort_values(by='final_measure', ascending=False).head(10)

# Buat grafik jaringan dengan pyvis
net = Network(notebook=True, height="750px", width="100%")

# Tentukan skala ukuran node
scale_factor_top = 30
scale_factor_default = 15

# Tambahkan node ke dalam grafik dengan ukuran berdasarkan pengaruh
for index, row in centrality_df.iterrows():
    node = row['account_name']
    size = scale_factor_top if node in top_10_final_measure['account_name'].values else scale_factor_default
    net.add_node(node, size=size, title=node)

# Tambahkan edge ke dalam grafik
for index, row in weighted_df.iterrows():
    net.add_edge(row['source'], row['target'])

# Menampilkan grafik
net.show('network_graph.html')